In [1]:
# hide
# default_exp tests.core.test_compose
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Test compose

In [39]:
# export
import pytest
import os
import joblib
from IPython.display import display
import pandas as pd
import numpy as np
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch
from sklearn.preprocessing import FunctionTransformer

from block_types.core.compose import *
from block_types.core.block_types import Component, PandasComponent, PickleSaverComponent
from block_types.core.utils import PickleIO
from block_types.utils.utils import remove_previous_results
from block_types.core.data_conversion import DataConverter, PandasConverter

import block_types.config.bt_defaults as dflt
from block_types.utils.utils import check_last_part

In [3]:
#export
@pytest.fixture (name='column_transformer_data')
def column_transformer_data_fixture():
    return column_transformer_data()

@pytest.fixture (name='multi_split_data')
def multi_split_data_fixture():
    return multi_split_data()

## Parallel

### `find_last_fitted_model`

#### Using Sequential

##### First test / example

In [4]:
# export
from block_types.utils.dummies import make_pipe_fit1

def test_pipeline_find_last_fitted_model_seq_others ():
    path_results = 'test_pipeline_find_last_fitted_model_seq_start'
    remove_previous_results (path_results=path_results)
    
    # pipelines
    pipe1 = make_pipe_fit1 ()
    X = np.array([1,2,3]).reshape(-1,1)
    r1 = pipe1.fit_apply (X)
    
    # case 1: component A
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    with pytest.raises (RuntimeError):
        r2 = pipe2.fit_apply (None)
    remove_previous_results (path_results=path_results)
    
    # case 2: component B
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)
    
    # case 2: component C
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    r = pipe2.C.fit_apply (r)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    pipe2.B.raise_error = True
    pipe2.B.estimator = Bunch ()
    pipe2.C.estimator = Bunch ()
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)

In [5]:
tst.run (test_pipeline_find_last_fitted_model_seq_others, tag='dummy')

running test_pipeline_find_last_fitted_model_seq_others


applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting pipeline (using whole data)
applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting B (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
applying B (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/B_result.pk
fitting pipeline (using whole data)
fitting B (using whole data)
loading from /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
loaded pre-trained B
applying B (on whole data)
loading from /home/jcidatascience/jaume/workspace/

##### Second test / example

#### Using Parallel

In [6]:
# export
from block_types.utils.dummies import make_pipe_fit2

def test_pipeline_find_last_fitted_model_parallel_2 ():
    path_results = 'test_pipeline_find_last_fitted_model_parallel_2'
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    # pipelines
    pipe1 = make_pipe_fit2 ()
    X = np.array([1,2,3]).reshape(-1,1)
    r1 = pipe1.fit_apply (X)
    
    # ******************************************************
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    b1 = pipe2.obj.B1.fit_apply (r)
    b2 = pipe2.obj.B2.fit_apply (r)
    
    b3a = pipe2.obj.B3a.fit_apply (r)
    b3b = pipe2.obj.B3b.fit_apply (b3a)
    b3c = pipe2.obj.B3c.fit_apply (b3b)
    b3d = pipe2.obj.B3d.fit_apply (b3c)
    
    b4a = pipe2.obj.B4a.fit_apply (r)
    b4b = pipe2.obj.B4b.fit_apply (b4a)
    b4c = pipe2.obj.B4c.fit_apply (b4b)
    b4d = pipe2.obj.B4d.fit_apply (b4c)
    b4e = pipe2.obj.B4e.fit_apply (b4d)
    
    b5 = pipe2.obj.B5.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    b1 = pipe2.obj.B1.fit_apply (r)
    b2 = pipe2.obj.B2.fit_apply (r)
    
    b3a = pipe2.obj.B3a.fit_apply (r)
    b3b = pipe2.obj.B3b.fit_apply (b3a)
    b3c = pipe2.obj.B3c.fit_apply (b3b)
    b3d = pipe2.obj.B3d.fit_apply (b3c)
    
    b4 = pipe2.obj.pipeline_1_1.fit_apply (r)
    b5 = pipe2.obj.B5.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    r = pipe2.obj.parallel.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B1.raise_error = True
    pipe2.obj.B2.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
    pipe2.obj.B3d.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    pipe2.obj.B4e.raise_error = True
    
    pipe2.obj.B5.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True, new_parallel=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    r = pipe2.obj.new_parallel.fit_apply (r)
    r = pipe2.C.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B1.raise_error = True
    pipe2.obj.B2.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
    pipe2.obj.B3d.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    pipe2.obj.B4e.raise_error = True
    
    pipe2.obj.B5.raise_error = True
    
    pipe2.new_parallel.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True, new_parallel=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    r = pipe2.obj.new_parallel.fit_apply (r)
    r = pipe2.C.fit_apply (r)
    r = pipe2.D.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B1.raise_error = True
    pipe2.obj.B2.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
    pipe2.obj.B3d.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    pipe2.obj.B4e.raise_error = True
    
    pipe2.obj.B5.raise_error = True
    
    pipe2.new_parallel.raise_error = True
    pipe2.C.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.D.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)

In [7]:
tst.run (test_pipeline_find_last_fitted_model_parallel_2, tag='dummy')

running test_pipeline_find_last_fitted_model_parallel_2
parallel class: <class 'block_types.core.compose.Parallel'>


applying A0 (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/A0_result.pk
fitting A1 (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/A1_estimator.pk
applying A1 (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/A1_result.pk
applying B1 (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B1_result.pk
fitting B2 (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B2_estimator.pk
applying B2 (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B2_re

parallel class: <class 'block_types.core.compose.Parallel'>


saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3a_result.pk
fitting B3b (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B3b_estimator.pk
applying B3b (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3b_result.pk
applying B3c (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3c_result.pk
fitting B3d (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B3d_estimator.pk
applying B3d (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3d_result.pk
applying 

parallel class: <class 'block_types.core.compose.Parallel'>


...

applying B3b (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3b_result.pk
applying B3c (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3c_result.pk
fitting B3d (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B3d_estimator.pk
applying B3d (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3d_result.pk
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/pipeline_1_result.pk
fitting pipeline_1_1 (using whole data)
applying B4a (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model

parallel class: <class 'block_types.utils.dummies.NewParallel'>


saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B3b_estimator.pk
applying B3b (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3b_result.pk
applying B3c (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3c_result.pk
fitting B3d (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B3d_estimator.pk
applying B3d (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3d_result.pk
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/pipeline_1_result.pk
fitting pipeline_1_1 (using whole

parallel class: <class 'block_types.utils.dummies.NewParallel'>


applying B3b (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3b_result.pk
applying B3c (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3c_result.pk
fitting B3d (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/models/B3d_estimator.pk
applying B3d (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/B3d_result.pk
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_parallel_2/whole/pipeline_1_result.pk
fitting pipeline_1_1 (using whole data)
applying B4a (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model

## Data conversion

In [8]:
#export
from block_types.utils.dummies import (DataSource, SumXY, MaxOfPositiveWithSeparateLabels, Sum1direct,
                                       Multiply10direct, subtract_xy, MinOfPositiveWithoutSeparateLabels)

In [12]:
result = DataSource ().apply()

In [13]:
result[0]

,a,b,c,d
0,1,4,10,40
1,2,5,20,50
2,3,6,30,60
3,4,7,40,70


In [14]:
result[1]

,a,b,c,d
0,2,5,1,4
1,3,6,2,5
2,4,7,3,6
3,5,8,4,7


In [12]:
SumXY (convert_before=lambda X: (X[0], X[1])).apply (result)

,a,b,c,d
0,3,9,11,44
1,5,11,22,55
2,7,13,33,66
3,9,15,44,77


In [23]:
class SumXYConverter (DataConverter):
    def convert_before_transforming (self, X, **kwargs):
        self.label = X[2]
        return X[0], X[1]
    def convert_after_transforming (self, result, **kwargs):
        result['label'] = self.label
        return result

In [25]:
pipe = Sequential (DataSource (),
                   SumXY (data_converter=SumXYConverter),
                   PandasComponent (apply=lambda X: X*2))
result = pipe ()
result

,a,b,c,d,label
0,6,18,22,88,0
1,10,22,44,110,1
2,14,26,66,132,1
3,18,30,88,154,0


In [26]:
result2 = pipe.fit_apply (())
result2

,a,b,c,d,label
0,6,18,22,88,0
1,10,22,44,110,1
2,14,26,66,132,1
3,18,30,88,154,0


In [30]:
result3=MaxOfPositiveWithSeparateLabels (data_converter='PandasConverter').fit_apply (result2)
result3

,a,b,c,d,label
0,74,206,286,1012,0
1,114,246,506,1232,1
2,154,286,726,1452,1
3,194,326,946,1672,0


In [34]:
parallel = Parallel (Sum1direct (data_converter='PandasConverter'), 
                     Multiply10direct (data_converter='PandasConverter'),
                     finalize_result=lambda X: tuple(X)
                    )
result4=parallel.fit_apply (result3)
result4

(     a    b    c     d  label
 0   75  207  287  1013      0
 1  115  247  507  1233      1
 2  155  287  727  1453      1
 3  195  327  947  1673      0,
       a     b     c      d  label
 0   740  2060  2860  10120      0
 1  1140  2460  5060  12320      1
 2  1540  2860  7260  14520      1
 3  1940  3260  9460  16720      0)

In [36]:
result5=PandasComponent(apply=subtract_xy).fit_apply (result4)
result5

,a,b,c,d,label
0,-665,-1853,-2573,-9107,0
1,-1025,-2213,-4553,-11087,0
2,-1385,-2573,-6533,-13067,0
3,-1745,-2933,-8513,-15047,0


In [40]:
result6=MinOfPositiveWithoutSeparateLabels (
    data_converter=PandasConverter (separate_labels=False)).fit_apply (result5)
result6

TypeError: _fit() takes 2 positional arguments but 3 were given

In [ ]:
pipe = Sequential (DataSource (),
                   SumXY (data_converter=SumXYConverter),
                   PandasComponent (apply=lambda X: X*2),
                   MaxOfPositiveWithSeparateLabels (data_converter='PandasConverter'),
                   Parallel (Sum1direct (data_converter='PandasConverter'), 
                             Multiply10direct (data_converter='PandasConverter'),
                             finalize_result=lambda X: tuple(X)),
                   PandasComponent(apply=subtract_xy),
                   MinOfPositiveWithoutSeparateLabels (),
                   make_column_transformer ((Multiply10direct (), ['a','b']), 
                                            (Sum1direct (), ['c','d'])),
                   Sum1direct ())